In [ ]:
f₂ = x -> abs(x)^3
df₂ = x -> x < 0 ? -3x^2 : 3x^2
ddf₂ = x -> x < 0 ? -6x : 6x
n = 101
x = cos.(π*(0:n)/n)
norm(df₂.(x) - chebfft(f₂.(x)),Inf)
norm(ddf₂.(x) - chebfft(chebfft(f₂.(x))),Inf)

In [ ]:
f₂ = x -> exp(-200*x^2)
df₂ = x -> -400*x*f₂(x)
ddf₂ = x -> -400*f₂(x) + 400^2*x^2*f₂(x)
n = 161
x = cos.(π*(0:n)/n)
@show norm(df₂.(x) - chebfft(f₂.(x)),Inf)
norm(ddf₂.(x) - chebfft(chebfft(f₂.(x))),Inf)

In [ ]:
n = 100
g = θ -> abs(cos(θ)) 
θ = range(0,2π;length= 2n+2)[1:end-1] 
norm(ifft(fft(g.(θ))) - g.(θ),Inf)

How does the rate of convergence of a Chebyshev interpolant of a periodic function compare to the rate of convergence of Fourier series?

In [ ]:
n = 161
x = cos.(π*(0:n)/n) 
τ = 1/n^2
τ = 0.001
u = exp.(-200*x.^2)
uold = exp.(-200*(x .- τ).^2)
T = 0.1
tplot = 0.01
plotgap = Int64(round(tplot/τ))
τ = tplot/plotgap
nplots = @show Int64(round(T/tplot))
plotdata = [transpose(u);zeros(nplots,n+1)]
tdata = [0.0]
for i = 1:nplots
    for n = 1:plotgap
        d2u = chebfft(chebfft(u))
        d2u[1] = d2u[n+1] = 0
        unew = 2u - uold + τ^2*d2u
        uold = u
        u = unew
    end
    plotdata[i+1,:] = u
    tdata = [tdata;τ*i*plotgap]
end

In [ ]:
surface(x,tdata,plotdata;seriescolor=:redsblues, camera=(10,50))

In [ ]:
n = 201
x = cos.(π*(0:n)/n) 
τ = 8/n^2
u = exp.(-200*x.^2)
uold = exp.(-200*(x .- τ).^2)
T = 0.5
tplot = 0.075
plotgap = Int64(round(tplot/τ))
τ = tplot/plotgap
nplots = Int64(round(T/tplot))
plotdata = [transpose(u);zeros(nplots,n+1)]
tdata = [0.0]
for i = 1:nplots
    for n = 1:plotgap
        d2u = chebfft(chebfft(u))
        d2u[1] = d2u[n+1] = 0
        unew = 2u - uold + τ^2*d2u
        uold = u
        u = unew
    end
    plotdata[i+1,:] = u
    tdata = [tdata;τ*i*plotgap]
end

In [1]:
function chebfft(f)
    n = length(f)-1
    x = cos.((0:n)*π/n) # Chebyshev points
    ii = 0:n-1
    q = [f; f[n:-1:2]]      # transform x -> θ    
    # differentiate the interpolant qₙ in coefficient space and map back to function values
    cq = real.(fft(q))
    dq = real.(ifft(im*[ii; 0; 1-n:-1] .*cq))
    df = zeros(n+1,1)
    # Compute approximations to f' at the interior points
    df[2:n] = -dq[2:n] ./sqrt.(1 .- x[2:n].^2)    # transform θ -> x   
    # At the boundary points
    df[1] = sum(ii.^2 .*cq[1:n])/n + .5*n*cq[n+1]     
    df[n+1] = sum((-1) .^(ii .+1) .* ii.^2 .*cq[1:n])/n +
              .5*(-1)^(n+1)*n*cq[n+1]
    df
end

chebfft (generic function with 1 method)

In [3]:
using LinearAlgebra, FFTW, Plots

In [5]:
n = 80
x = cos.(π*(0:n)/n) 
τ = 8/n^2
u = exp.(-200*x.^2)
chebfft(chebfft(u))

81×1 Matrix{Float64}:
 18.814813387204527
  1.43502310819898
 -0.3623414922289144
  0.16372246796671705
 -0.09423656545353788
  0.062108847303584055
 -0.044693987341527484
  0.034233232427259515
 -0.02748620939502835
  0.02290550068372511
 -0.01967675708965887
  0.017338686924811633
 -0.015614564507098527
  ⋮
  0.01733868692472177
 -0.019676757089406503
  0.022905500683500737
 -0.027486209394875238
  0.03423323242725595
 -0.04469398734150329
  0.06210884730329245
 -0.09423656545328828
  0.16372246796641848
 -0.36234149222556905
  1.4350231081945835
 18.81481338718346

In [6]:
 Time-stepping by leap frog formula:% Time-stepping by leap frog formula:
  N = 80; x = cos(pi*(0:N)/N); dt = 8/N^2;
  v = exp(-200*x.^2); vold = exp(-200*(x-dt).^2);
  tmax = 4; tplot = .075; 
  plotgap = round(tplot/dt); dt = tplot/plotgap;
  nplots = round(tmax/tplot);
  plotdata = [v; zeros(nplots,N+1)]; tdata = 0;
  clf, drawnow, h = waitbar(0,'please wait...');
  set(gcf,'renderer','zbuffer')
  
  M = 2;
  [x, DM] = chebdif(N+1, M);
  D2 = DM(2:N,2:N,2);
  
  tic
  for i = 1:nplots, waitbar(i/nplots)
    for n = 1:plotgap
        % FFT
      w = chebfft(chebfft(v))'; w(1) = 0; w(N+1) = 0;
      vnew = 2*v - vold + dt^2*w; vold = v; v = vnew;
      % Differentiation matrix
      %vnew = 2*v - vold + [0, dt^2*(D2*v(2:end-1)')', 0]; vold = v; v = vnew;
    end
    plotdata(i+1,:) = v; tdata = [tdata; dt*i*plotgap];
  end
  toc
  N = 80; x = cos(pi*(0:N)/N); dt = 8/N^2;
  v = exp(-200*x.^2); vold = exp(-200*(x-dt).^2);
  tmax = 4; tplot = .075; 
  plotgap = round(tplot/dt); dt = tplot/plotgap;
  nplots = round(tmax/tplot);
  plotdata = [v; zeros(nplots,N+1)]; tdata = 0;
  clf, drawnow, h = waitbar(0,'please wait...');
  set(gcf,'renderer','zbuffer')
  
  M = 2;
  [x, DM] = chebdif(N+1, M);
  D2 = DM(2:N,2:N,2);
  
  tic
  for i = 1:nplots, waitbar(i/nplots)
    for n = 1:plotgap
        % FFT
      w = chebfft(chebfft(v))'; w(1) = 0; w(N+1) = 0;
      vnew = 2*v - vold + dt^2*w; vold = v; v = vnew;
      % Differentiation matrix
      %vnew = 2*v - vold + [0, dt^2*(D2*v(2:end-1)')', 0]; vold = v; v = vnew;
    end
    plotdata(i+1,:) = v; tdata = [tdata; dt*i*plotgap];
  end
  toc

LoadError: syntax: "%" is not a unary operator

In [ ]:
 Time-stepping by leap frog formula:
  N = 80; x = cos(pi*(0:N)/N); dt = 8/N^2;
  v = exp(-200*x.^2); vold = exp(-200*(x-dt).^2);
  tmax = 4; tplot = .075; 
  plotgap = round(tplot/dt); dt = tplot/plotgap;
  nplots = round(tmax/tplot);
  plotdata = [v; zeros(nplots,N+1)]; tdata = 0;
  clf, drawnow, h = waitbar(0,'please wait...');
  set(gcf,'renderer','zbuffer')
  
  M = 2;
  [x, DM] = chebdif(N+1, M);
  D2 = DM(2:N,2:N,2);
  
  tic
  for i = 1:nplots, waitbar(i/nplots)
    for n = 1:plotgap
        % FFT
      w = chebfft(chebfft(v))'; w(1) = 0; w(N+1) = 0;
      vnew = 2*v - vold + dt^2*w; vold = v; v = vnew;
      % Differentiation matrix
      %vnew = 2*v - vold + [0, dt^2*(D2*v(2:end-1)')', 0]; vold = v; v = vnew;
    end
    plotdata(i+1,:) = v; tdata = [tdata; dt*i*plotgap];
  end
  toc















In [ ]:
% Time-stepping by leap frog formula:
  N = 80; x = cos(pi*(0:N)/N); dt = 8/N^2;
  v = exp(-200*x.^2); vold = exp(-200*(x-dt).^2);
  tmax = 4; tplot = .075; 
  plotgap = round(tplot/dt); dt = tplot/plotgap;
  nplots = round(tmax/tplot);
  plotdata = [v; zeros(nplots,N+1)]; tdata = 0;
  clf, drawnow, h = waitbar(0,'please wait...');
  set(gcf,'renderer','zbuffer')
  
  M = 2;
  [x, DM] = chebdif(N+1, M);
  D2 = DM(2:N,2:N,2);
  
  tic
  for i = 1:nplots, waitbar(i/nplots)
    for n = 1:plotgap
        % FFT
      w = chebfft(chebfft(v))'; w(1) = 0; w(N+1) = 0;
      vnew = 2*v - vold + dt^2*w; vold = v; v = vnew;
      % Differentiation matrix
      %vnew = 2*v - vold + [0, dt^2*(D2*v(2:end-1)')', 0]; vold = v; v = vnew;
    end
    plotdata(i+1,:) = v; tdata = [tdata; dt*i*plotgap];
  end
  toc
















